# Abstract

Inspiradonme en los siguientes dataset y fuentes, quise aprovechar la bases de datos, para realizar un proyecto similar pero con Cancer.

https://www.kaggle.com/datasets/cdc/behavioral-risk-factor-surveillance-system
https://www.kaggle.com/code/alexteboul/heart-disease-health-indicators-dataset-notebook
https://www.kaggle.com/datasets/alexteboul/heart-disease-health-indicators-dataset/data

Para eso utilice la fuente mas actual: https://www.cdc.gov/brfss/annual_data/annual_2022.html

La idea es realizar un dataset y luego poder aplicarle diferentes algoritmos,e intentar de hacer un modelo predictivo de Cancer.





# Instalación y  importacion de librerias

In [56]:
!pip install pyreadstat


[notice] A new release of pip is available: 24.1.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [57]:
import requests
import re
import zipfile
import os
from urllib.parse import urlsplit
import numpy as np
import pandas as pd
import glob
import pyreadstat
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score


# Creacion del Data Frame

In [58]:
# URL del archivo CSV
url = 'https://www.cdc.gov/brfss/annual_data/2022/files/LLCP2022XPT.zip'

# Realizar la solicitud GET
response = requests.get(url, allow_redirects=True)

# Intentar obtener el nombre del archivo desde el encabezado Content-Disposition
content_disposition = response.headers.get('Content-Disposition')
if content_disposition:
    filename = re.findall('filename="(.+)"', content_disposition)
    if filename:
        filename = filename[0]
    else:
        # Si no se encuentra el nombre en Content-Disposition, usar el último segmento de la URL
        filename = urlsplit(url).path.split('/')[-1]
else:
    # Si no hay encabezado Content-Disposition, usar el último segmento de la URL
    filename = urlsplit(url).path.split('/')[-1]

# Descargar y guardar el archivo
with open(filename, 'wb') as file:
    file.write(response.content)

print(f"Archivo descargado y guardado como {filename}.")

# Directorio donde se extraerán los archivos
extract_dir = 'extraido/'

# Crear el directorio si no existe
os.makedirs(extract_dir, exist_ok=True)

# Descomprimir el archivo ZIP usando el nombre obtenido automáticamente
with zipfile.ZipFile(filename, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print(f"Archivo ZIP descomprimido en {extract_dir}.")



Archivo descargado y guardado como LLCP2022XPT.zip.
Archivo ZIP descomprimido en extraido/.


In [59]:
extract_dir = 'extraido/'  # Asegúrate de que esta ruta sea correcta

# Listar todos los archivos en el directorio
archivos = os.listdir(extract_dir)
print(f"Archivos en el directorio '{extract_dir}': {archivos}")

Archivos en el directorio 'extraido/': ['LLCP2022.csv', 'LLCP2022.XPT']


In [60]:
# Listar y limpiar nombres de archivos en el directorio
archivos = os.listdir(extract_dir)
for archivo in archivos:
    archivo_limpio = archivo.strip()  # Eliminar espacios en blanco al inicio y final
    if archivo != archivo_limpio:
        os.rename(os.path.join(extract_dir, archivo), os.path.join(extract_dir, archivo_limpio))
        print(f"Renombrado: '{archivo}' a '{archivo_limpio}'")

# Verificar que los archivos hayan sido renombrados correctamente
archivos_limpios = os.listdir(extract_dir)
print(f"Archivos en el directorio '{extract_dir}' después de limpiar: {archivos_limpios}")


Archivos en el directorio 'extraido/' después de limpiar: ['LLCP2022.csv', 'LLCP2022.XPT']


In [61]:
# Directorio donde se extrajeron los archivos
extract_dir = 'extraido/'

# Buscar archivos .XPT en el directorio extraído
xpt_files = glob.glob(os.path.join(extract_dir, '*.XPT'))

# Verificar si se encontraron archivos .XPT
if xpt_files:
    print(f"Archivos .XPT encontrados: {xpt_files}")


else:
    print("No se encontraron archivos .XPT en el directorio extraído.")




Archivos .XPT encontrados: ['extraido\\LLCP2022.XPT']


In [62]:

# Intentar leer el archivo .XPT directamente con pandas
try:
    df = pd.read_sas('extraido/LLCP2022.XPT', format='xport', encoding='latin1')  # Intenta con 'latin1'
    csv_file = 'extraido/LLCP2022.csv'
    df.to_csv(csv_file, index=False)
    print(f"Archivo 'extraido/LLCP2022.XPT' convertido a {csv_file}")
except Exception as e:
    print(f"Error al convertir 'extraido/LLCP2022.XPT' a CSV: {e}")

KeyboardInterrupt: 

In [ ]:
# Buscar archivos .csv en el directorio extraído
csv_files = glob.glob(os.path.join(extract_dir, '*.csv'))

if csv_files:
    print(f"Archivos .CSV encontrados: {csv_files}")

    # Tomar el primer archivo CSV encontrado
    csv_file = csv_files[0]

    # Leer el archivo CSV con pandas
    df = pd.read_csv(csv_file)

    # Mostrar las primeras filas del DataFrame
    print(f"Archivo CSV encontrado: {csv_file}")
    print("Contenido del archivo CSV:")
    print(df.head())
else:
    print("No se encontraron archivos CSV en el directorio extraído.")

# Selección y limpieza de datos

In [ ]:
df

La Tabla es demaciado grande (445132 rows × 328 columns). Ademas de que mucha informacion no es relevante para el proyecto. Para limpieza y seleccion de los mismos se utiliza el Codebook respectivo a la base de datos original. Y debido a su gran tamaño realice un segundo notebook para analizarlo por separado:
https://colab.research.google.com/drive/1k_iYvFPBALUv31XEwnT6MCnU3Kjwc0op#scrollTo=UJBc4YDu7us3

En este segundo notebook se explica la seleción de las diferentes columnas.

In [ ]:
# Lista de columnas que quieres filtrar
columnas_para_filtrar = [
    'SMOKE100', '_BMI5', '_RFDRHV8',  '_TOTINDA', '_AGEG5YR', 'TRNSGNDR',
    'CHCSCNC1', 'CHCOCNC1', 'HADHYST2', 'CNCRDIFF', 'CNCRAGE', 'CNCRTYP2',  '_SEX', '_PHYS14D', '_MENT14D'
]
# Filtra el DataFrame para incluir solo las columnas especificadas
df_filtrado = df[columnas_para_filtrar]

print("DataFrame filtrado:")
df_filtrado

Una vez  seleccionadas las columnas relevantes, paso a trabajar y preparar las mismas. En base al Codebook

In [ ]:
df_filtrado.isna().sum()

In [ ]:
#Los vacios de si fuma los lleno con No sabe(7) por no saber el resultado
df_filtrado['SMOKE100'] = df_filtrado['SMOKE100'].fillna(7)
df_filtrado['Fuma?'] = df_filtrado['SMOKE100'].map({
    1: 'Si',
    2: 'No',
    7: 'No sabe',
    9: 'Se Rehusa a responder'
}).fillna('Faltante')
df_filtrado = df_filtrado.rename(columns={'SMOKE100': 'SMOKE100>Fuma?'})

In [ ]:
df_filtrado = df_filtrado.dropna(subset=['_BMI5'])
df_filtrado.loc[:, '_BMI5'] = df_filtrado['_BMI5'].astype(int)
df_filtrado = df_filtrado.rename(columns={'_BMI5': 'IMC'})

In [ ]:
df_filtrado['Toma_mucho_Alcohol?'] = df_filtrado['_RFDRHV8'].map({
    1: 'No',
    2: 'Si',
    9: 'No se sabe'
}).fillna('Faltante')
df_filtrado = df_filtrado.rename(columns={'_RFDRHV8': '_RFDRHV8>Toma_mucho_Alcohol?'})

In [ ]:
df_filtrado['Acitividad_Fisica?'] = df_filtrado['_TOTINDA'].map({
    1: 'Si',
    2: 'No',
    9: 'No se sabe'
}).fillna('Faltante')
df_filtrado = df_filtrado.rename(columns={'_TOTINDA': '_TOTINDA>Acitividad_Fisica?'})

In [ ]:
df_filtrado['Edad'] = df_filtrado['_AGEG5YR'].map({
    1: 'De 18 a 24',
    2: 'De 25 a 29',
    3: 'De 30 a 34',
    4: 'De 35 a 39',
    5: 'De 40 a 44',
    6: 'De 45 a 49',
    7: 'De 50 a 54',
    8: 'De 55 a 59',
    9: 'De 60 a 64',
    10: 'De 65 a 69',
    11: 'De 70 a 74',
    12: 'De 75 a 79',
    13: 'De 80 o más'
}).fillna('Desconocido')
df_filtrado = df_filtrado.rename(columns={'_AGEG5YR': '_AGEG5YR>Edad?'})

In [ ]:
#Teniendo en cuenta que la poblacion Trans es menor al 1%, se asume que los datos vacios son No(4)
df_filtrado['TRNSGNDR'] = df_filtrado['TRNSGNDR'].fillna('4')
df_filtrado['TRANS?'] = df_filtrado['TRNSGNDR'].map({
    1: 'Sí, Transgénero, masculino a femenino',
    2: 'Sí, Transgénero, femenino a masculino',
    3: 'Sí, Transgénero, no conforme con el género',
    4: 'No',
    7: 'No sabe/No está seguro',
    9: 'Se rehúsa a responder',
}).fillna('No')
df_filtrado = df_filtrado.rename(columns={'TRNSGNDR': 'TRNSGNDR>TRANS?'})

In [ ]:
df_filtrado['Cancer de piel diferente a melanoma'] = df_filtrado['CHCSCNC1'].map({
    1: 'Sí',
    2: 'No',
    7: 'No sabe / No está seguro',
    9: 'Se rehúsa a responder',
    'BLANK': 'No preguntado o Faltante'
}).fillna('No preguntado o Faltante')
df_filtrado = df_filtrado.rename(columns={'CHCSCNC1': 'CHCSCNC1>Cancer de piel diferente a melanoma?'})

In [ ]:
df_filtrado['CHCOCNC1'] = df_filtrado['CHCOCNC1'].fillna(2)
df_filtrado['Melanoma u otro cancer'] = df_filtrado['CHCOCNC1'].map({
    1: 'Sí',
    2: 'No',
    7: 'No sabe / No está seguro',
    9: 'Se rehúsa a responder',
    'BLANK': 'No preguntado o Faltante'
}).fillna('No preguntado o Faltante')
df_filtrado = df_filtrado.rename(columns={'CHCOCNC1': 'CHCOCNC1>Melanoma u otro cancer?'})

In [ ]:
#Un hombre no puede tener una histerectomía, por lo que deberia ser un No(2) directo

df_filtrado['HADHYST2'] = np.where(
    (df_filtrado['_SEX'] == 1) & (df_filtrado['HADHYST2'].isna()),
    2,
    df_filtrado['HADHYST2']
)

# Luego, llenar los vacíos restantes con 7 ya que no se sabe
df_filtrado['HADHYST2'] = df_filtrado['HADHYST2'].fillna(7)


In [ ]:
df_filtrado['histerectomía?'] = df_filtrado['HADHYST2'].map({
    1: 'Sí',
    2: 'No',
    7: 'No sabe/No está seguro',
    9: 'Se rehúsa a responder',
    'BLANK': 'No'
}).fillna('No')
df_filtrado = df_filtrado.rename(columns={'HADHYST2': 'HADHYST2>histerectomía?'})

In [ ]:
#No esta aclaro los que no tuvo, por lo que vacios se contemplan como que no tuvo(0)
df_filtrado['CNCRDIFF'] = df_filtrado['CNCRDIFF'].fillna(0)
df_filtrado['Cuantos_tipos_de_cancer_tuvo?'] = df_filtrado['CNCRDIFF'].map({
    1: 'Solo uno',
    2: 'Dos',
    3: 'Tres o más',
    7: 'No sabe',
    9: 'Se rehúsa a responder',
    0: 'No tuvo'
}).fillna('No tuvo')
df_filtrado = df_filtrado.rename(columns={'CNCRDIFF': 'CNCRDIFF>Cuantos_tipos_de_cancer_tuvo?'})

In [ ]:
# CNCRAGE corresponde a la edad cuando se enteraron que tenian cancer se utiliza 200 para indicar que no tuvo
df_filtrado['CNCRAGE'] = df_filtrado['CNCRAGE'].fillna(200)
df_filtrado['CNCRAGE'] = df_filtrado['CNCRAGE'].astype(int)
df_filtrado = df_filtrado.rename(columns={'CNCRAGE': 'CNCRAGE>Con_Cuantos_años_se_entero_del_cancer?'})

In [ ]:
#Igual que en los anteriores se llenan los vacios para indicar que no tuvieron cancer
df_filtrado['CNCRTYP2'] = df_filtrado['CNCRTYP2'].fillna(0)
df_filtrado['Que_tipo_de_cancer?'] = df_filtrado['CNCRTYP2'].map({
    0: 'Ninguno',
    1: 'Vejiga',
    2: 'Sangre',
    3: 'Hueso',
    4: 'Cerebro',
    5: 'Mama',
    6: 'Cérvix/Cervical',
    7: 'Colon',
    8: 'Esófago/Esófagico',
    9: 'Vesícula biliar',
    10: 'Riñón',
    11: 'Laringe-tráquea',
    12: 'Leucemia',
    13: 'Hígado',
    14: 'Pulmón',
    15: 'Linfoma',
    16: 'Melanoma',
    17: 'Boca/lengua/labio',
    18: 'Ovario/Ovárico',
    19: 'Páncreas/Pancreático',
    20: 'Próstata',
    21: 'Recto/Rectal',
    22: 'Piel (no melanoma)',
    23: 'Piel (no se sabe qué tipo)',
    24: 'Tejido blando (músculo o grasa)',
    25: 'Estómago',
    26: 'Testículo/Testicular',
    27: 'Garganta - faringe',
    28: 'Tiroides',
    29: 'Útero/Uterino',
    30: 'Otro',
    77: 'No sabe/No está seguro',
    99: 'Se rehúsa a responder',
    'BLANK': 'No preguntado o Faltante'
}).fillna('No preguntado o Faltante')
df_filtrado = df_filtrado.rename(columns={'CNCRTYP2': 'CNCRTYP2>Que_tipo_de_cancer?'})

In [ ]:
df_filtrado['Sexo'] = df_filtrado['_SEX'].map({
    1: 'Masculino',
    2: 'Femenino'
}).fillna('No especificado')


In [ ]:
df_filtrado['Estado_físico_no_bueno_ultimo_tiempo'] = df_filtrado['_PHYS14D'].map({
    1: 'Cero días con salud física no buena',
    2: '1-13 días con salud física no buena',
    3: '14+ días con salud física no buena',
    9: 'No sabe/Se rehúsa/Faltante'
}).fillna('No especificado')
df_filtrado = df_filtrado.rename(columns={'_PHYS14D': '_PHYS14D>Estado_físico_no_bueno_ultimo_tiempo'})

In [ ]:
df_filtrado['Estado_mental_no_bueno_ultimo_tiempo'] = df_filtrado['_MENT14D'].map({
    1: 'Cero días con salud mental no buena',
    2: '1-13 días con salud mental no buena',
    3: '14+ días con salud mental no buena',
    9: 'No sabe/Se rehúsa/Faltante'
}).fillna('No especificado')
df_filtrado = df_filtrado.rename(columns={'_MENT14D': '_MENT14D>Estado_mental_no_bueno_ultimo_tiempo?'})

In [ ]:
df_filtrado.isna().sum()

In [ ]:
df_filtrado

In [ ]:

df_filtrado2 = df_filtrado[['Sexo','_AGEG5YR>Edad?','IMC', 'Fuma?',
       'Toma_mucho_Alcohol?', 'Acitividad_Fisica?', 'Edad', 'TRANS?',
       'Cancer de piel diferente a melanoma', 'Melanoma u otro cancer',
       'histerectomía?', 'Cuantos_tipos_de_cancer_tuvo?',
       'Que_tipo_de_cancer?',  'Estado_físico_no_bueno_ultimo_tiempo',
       'Estado_mental_no_bueno_ultimo_tiempo']]
df_filtrado2.loc[:, 'Tuvo_Cancer?'] = df_filtrado2['Que_tipo_de_cancer?'].apply(
    lambda x: 'No' if x in ['Ninguno', 'No sabe/No está seguro'] else 'Sí'
)


df_filtrado2

In [ ]:
df_analisis = df_filtrado2[['Tuvo_Cancer?','Sexo','IMC', 'Fuma?',
       'Toma_mucho_Alcohol?', 'Acitividad_Fisica?', 'Edad', 'TRANS?']]
df_analisis

# Analisis

In [ ]:
# Agrupar por 'IMC' y 'Tuvo_Cancer?', y contar las ocurrencias
df_grouped = df_analisis.groupby(['IMC', 'Tuvo_Cancer?']).size().unstack(fill_value=0)

# Crear el gráfico de líneas
plt.figure(figsize=(10, 6))
df_grouped.plot(kind='line', ax=plt.gca())

# Personalizar el eje x para tener 10 ticks
num_ticks = 20
plt.xticks(ticks=np.linspace(df_analisis['IMC'].min(), df_analisis['IMC'].max(), num_ticks))

plt.title('Conteo de Tuvo_Cancer? en función del IMC')
plt.xlabel('IMC')
plt.ylabel('Conteo')
plt.grid(True)
plt.show()

In [ ]:
# Filtrar el DataFrame para 'Tuvo_Cancer?'
imc_cancer_si = df_analisis[df_analisis['Tuvo_Cancer?'] == 'Sí']['IMC']
imc_cancer_no = df_analisis[df_analisis['Tuvo_Cancer?'] == 'No']['IMC']

# Calcular moda, media y mediana para 'Sí'
moda_imc_si = imc_cancer_si.mode()[0]
media_imc_si = imc_cancer_si.mean()
mediana_imc_si = imc_cancer_si.median()

# Calcular moda, media y mediana para 'No'
moda_imc_no = imc_cancer_no.mode()[0]
media_imc_no = imc_cancer_no.mean()
mediana_imc_no = imc_cancer_no.median()

# Mostrar los resultados
print("Resultados para 'Tuvo_Cancer? = Sí'")
print(f"Moda del IMC: {moda_imc_si}")
print(f"Media del IMC: {media_imc_si}")
print(f"Mediana del IMC: {mediana_imc_si}")

print("\nResultados para 'Tuvo_Cancer? = No'")
print(f"Moda del IMC: {moda_imc_no}")
print(f"Media del IMC: {media_imc_no}")
print(f"Mediana del IMC: {mediana_imc_no}")

El Índice de masa corporal no parece afectar en principio la aparición de cancer. Podemos verlo a nivel individual de cada tipo, pero no todos los tipos de cancer tienen una cantidad relevante por lo que se filtrara en base a la ocurrencia en la base de datos:


In [ ]:
conteo_cancer = df_filtrado2['Que_tipo_de_cancer?'].value_counts()
print(conteo_cancer)

In [ ]:
# Lista de tipos de cáncer que quieres filtrar
tipos_de_cancer_filtrados = ['Ninguno', 'Piel (no melanoma)', 'Mama', 'Melanoma', 'Próstata', 'Piel (no se sabe qué tipo)']

# Iterar sobre cada valor único en la columna 'Que_tipo_de_cancer?'
for cancer_type in tipos_de_cancer_filtrados:
    # Filtrar el DataFrame para el tipo de cáncer actual
    df_cancer_type = df_filtrado2[df_filtrado2['Que_tipo_de_cancer?'] == cancer_type]

    # Agrupar por 'IMC' y 'Tuvo_Cancer?', y contar las ocurrencias
    df_grouped = df_cancer_type.groupby(['IMC', 'Tuvo_Cancer?']).size().unstack(fill_value=0)

    # Calcular moda, media y mediana para 'Sí'
    imc_cancer_si = df_cancer_type[df_cancer_type['Tuvo_Cancer?'] == 'Sí']['IMC']
    moda_imc_si = imc_cancer_si.mode()[0] if not imc_cancer_si.empty else np.nan
    media_imc_si = imc_cancer_si.mean() if not imc_cancer_si.empty else np.nan
    mediana_imc_si = imc_cancer_si.median() if not imc_cancer_si.empty else np.nan

    # Calcular moda, media y mediana para 'No'
    imc_cancer_no = df_cancer_type[df_cancer_type['Tuvo_Cancer?'] == 'No']['IMC']
    moda_imc_no = imc_cancer_no.mode()[0] if not imc_cancer_no.empty else np.nan
    media_imc_no = imc_cancer_no.mean() if not imc_cancer_no.empty else np.nan
    mediana_imc_no = imc_cancer_no.median() if not imc_cancer_no.empty else np.nan

    # Crear el gráfico de líneas
    plt.figure(figsize=(10, 6))
    df_grouped.plot(kind='line', ax=plt.gca())

    # Personalizar el eje x para tener 20 ticks
    num_ticks = 20
    plt.xticks(ticks=np.linspace(df_analisis['IMC'].min(), df_analisis['IMC'].max(), num_ticks))

    # Añadir moda, media y mediana en el gráfico
    textstr = f"Tuvo_Cancer? = Sí\nModa: {moda_imc_si:.2f}\nMedia: {media_imc_si:.2f}\nMediana: {mediana_imc_si:.2f}"
    textstr += f"\n\nTuvo_Cancer? = No\nModa: {moda_imc_no:.2f}\nMedia: {media_imc_no:.2f}\nMediana: {mediana_imc_no:.2f}"

    plt.text(0.05, 0.95, textstr, transform=plt.gca().transAxes, fontsize=10,
             verticalalignment='top', bbox=dict(facecolor='white', alpha=0.5))

    plt.title(f'Conteo de Tuvo_Cancer? para {cancer_type} en función del IMC')
    plt.xlabel('IMC')
    plt.ylabel('Conteo')
    plt.grid(True)
    plt.show()


No parece que el IMC tenga incidencia en los casos de cancer, por lo menos en los tipos de cancer vistos, podria analisarse otros pero seria necesario una mayor cantidad de casos positivos en la muestra.

In [ ]:
# Agrupar por 'Edad' y 'Tuvo_Cancer?', y contar las ocurrencias
df_grouped = df_analisis.groupby(['Edad', 'Tuvo_Cancer?']).size().unstack(fill_value=0)

# Crear un gráfico de barras usando seaborn
plt.figure(figsize=(12, 8))
df_grouped.plot(kind='bar', figsize=(12, 8))

# Personalizar el gráfico
plt.title('Conteo de casos de Tuvo_Cancer? por Edad')
plt.xlabel('Edad')
plt.ylabel('Conteo')
plt.xticks(rotation=45)  # Rotar las etiquetas del eje x para mejor visibilidad
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.legend(title='Tuvo_Cancer?')
plt.tight_layout()  # Ajustar el diseño para que todo se muestre bien

plt.show()

En este caso es evidente que la aparición de Cancer aumenta con la edad.

In [ ]:
# Lista de tipos de cáncer que quieres filtrar
tipos_de_cancer_filtrados = ['Ninguno', 'Piel (no melanoma)', 'Mama', 'Melanoma', 'Próstata', 'Piel (no se sabe qué tipo)']

# Iterar sobre cada valor único en la lista de tipos de cáncer filtrados
for cancer_type in tipos_de_cancer_filtrados:
    # Filtrar el DataFrame para el tipo de cáncer actual
    df_cancer_type = df_filtrado2[df_filtrado2['Que_tipo_de_cancer?'] == cancer_type]

    # Agrupar por 'Edad' y 'Tuvo_Cancer?', y contar las ocurrencias
    df_grouped = df_cancer_type.groupby(['Edad', 'Tuvo_Cancer?']).size().unstack(fill_value=0)

    # Crear el gráfico de barras
    plt.figure(figsize=(12, 8))
    df_grouped.plot(kind='bar', stacked=True, figsize=(12, 8))

    # Personalizar el gráfico
    plt.title(f'Conteo de casos de Tuvo_Cancer? por Edad para {cancer_type}')
    plt.xlabel('Edad')
    plt.ylabel('Conteo')
    plt.xticks(rotation=45)  # Rotar las etiquetas del eje x para mejor visibilidad
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.legend(title='Tuvo_Cancer?')
    plt.tight_layout()  # Ajustar el diseño para que todo se muestre bien

    plt.show()


In [ ]:
# Filtrar el DataFrame para excluir los valores 'No sabe/No está seguro' y 'Se rehúsa a responder' en 'TRANS?'
df_filteredT = df_analisis[~df_analisis['TRANS?'].isin(['No sabe/No está seguro', 'Se rehúsa a responder'])]

# Agrupar el DataFrame por 'TRANS?' y 'Tuvo_Cancer?', y contar las ocurrencias
df_grouped = df_filteredT.groupby(['TRANS?', 'Tuvo_Cancer?']).size().unstack(fill_value=0)

# Crear gráficos de tortas para cada combinación de 'Tuvo_Cancer?'
for category in df_grouped.index:
    plt.figure(figsize=(8, 6))
    df_grouped.loc[category].plot(kind='pie', autopct='%1.1f%%', startangle=90)
    plt.title(f'Distribución de Tuvo_Cancer? para TRANS? = {category}')
    plt.ylabel('')  # Ocultar la etiqueta del eje y
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.tight_layout()  # Ajustar el diseño para que todo se muestre bien
    plt.show()

Por lo menos en casos de cancer en general pareciera ser de menor proporcion en personas Trans. Depende la bibliografia es acorde pero depende el tipo de cancer. Debido a la poca cantidad de muestra, al no ser representativa no se puede utilizar como indicador.

In [ ]:
# Filtrar el DataFrame para excluir los valores 'No sabe/No está seguro' y 'Se rehúsa a responder'
df_filteredT = df_analisis[~df_analisis['Fuma?'].isin(['No sabe', 'Se Rehusa a responder'])]

# Agrupar el DataFrame por 'TRANS?' y 'Tuvo_Cancer?', y contar las ocurrencias
df_grouped = df_filteredT.groupby(['Fuma?', 'Tuvo_Cancer?']).size().unstack(fill_value=0)

# Crear gráficos de tortas para cada combinación de 'Tuvo_Cancer?'
for category in df_grouped.index:
    plt.figure(figsize=(8, 6))
    df_grouped.loc[category].plot(kind='pie', autopct='%1.1f%%', startangle=90)
    plt.title(f'Distribución de Tuvo_Cancer? para Fuma? = {category}')
    plt.ylabel('')  # Ocultar la etiqueta del eje y
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.tight_layout()  # Ajustar el diseño para que todo se muestre bien
    plt.show()


In [ ]:
# Valores específicos para 'Que_tipo_de_cancer?'
valores_cancer = ['Ninguno', 'Piel (no melanoma)', 'Mama', 'Melanoma', 'Próstata', 'Piel (no se sabe qué tipo)' , 'Pulmón', 'Garganta - faringe']

# Filtrar el DataFrame para los valores específicos en 'Que_tipo_de_cancer?'
df_filtered_cancer = df_filtrado2[df_filtrado2['Que_tipo_de_cancer?'].isin(valores_cancer)]

# Filtrar el DataFrame para excluir los valores 'No sabe/No está seguro' y 'Se rehúsa a responder' en 'Fuma?'
df_filteredT = df_filtered_cancer[~df_filtered_cancer['Fuma?'].isin(['No sabe/No está seguro', 'Se rehúsa a responder'])]

# Agrupar el DataFrame por 'Fuma?' y contar las ocurrencias para cada tipo de cáncer
df_grouped = df_filteredT.groupby(['Que_tipo_de_cancer?', 'Fuma?']).size().unstack(fill_value=0)

# Crear gráficos de tortas para cada tipo de cáncer
for cancer_type in valores_cancer:
    df_cancer_type = df_grouped.loc[cancer_type] if cancer_type in df_grouped.index else pd.Series()

    if not df_cancer_type.empty and df_cancer_type.sum() > 0:
        plt.figure(figsize=(8, 6))
        df_cancer_type.plot(kind='pie', autopct='%1.1f%%', startangle=90)
        plt.title(f'Distribución de Fuma? para Tipo de Cáncer = {cancer_type}')
        plt.ylabel('')  # Ocultar la etiqueta del eje y
        plt.grid(axis='y', linestyle='--', alpha=0.7)
        plt.tight_layout()  # Ajustar el diseño para que todo se muestre bien
        plt.show()
    else:
        print(f'No hay datos para el Tipo de Cáncer = {cancer_type}')

Fumar pareciera que aumenta en general la probabilidad de cancer, pero en caso de los tipos de cancer mas especificos de personas que fuman ( 'Pulmón' y  'Garganta - faringe') la diferencia es enorme.

In [ ]:
# Filtrar el DataFrame para excluir los valores 'No sabe/No está seguro' y 'Se rehúsa a responder'
df_filteredT = df_analisis[~df_analisis['Toma_mucho_Alcohol?'].isin(['No se sabe', 'Se Rehusa a responder'])]

# Agrupar el DataFrame por 'TRANS?' y 'Tuvo_Cancer?', y contar las ocurrencias
df_grouped = df_filteredT.groupby(['Toma_mucho_Alcohol?', 'Tuvo_Cancer?']).size().unstack(fill_value=0)

# Crear gráficos de tortas para cada combinación de 'Tuvo_Cancer?'
for category in df_grouped.index:
    plt.figure(figsize=(8, 6))
    df_grouped.loc[category].plot(kind='pie', autopct='%1.1f%%', startangle=90)
    plt.title(f'Distribución de Tuvo_Cancer? para Toma_mucho_Alcohol? = {category}')
    plt.ylabel('')  # Ocultar la etiqueta del eje y
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.tight_layout()  # Ajustar el diseño para que todo se muestre bien
    plt.show()

    #Fuma?	Toma_mucho_Alcohol?	Acitividad_Fisica?

In [ ]:
# Valores específicos para 'Que_tipo_de_cancer?'
valores_cancer = ['Ninguno', 'Piel (no melanoma)', 'Mama', 'Melanoma', 'Próstata', 'Piel (no se sabe qué tipo)' , 'Pulmón', 'Garganta - faringe', 'Hígado']

# Filtrar el DataFrame para los valores específicos en 'Que_tipo_de_cancer?'
df_filtered_cancer = df_filtrado2[df_filtrado2['Que_tipo_de_cancer?'].isin(valores_cancer)]

# Filtrar el DataFrame para excluir los valores 'No sabe/No está seguro' y 'Se rehúsa a responder' en 'Fuma?'
df_filteredT = df_filtered_cancer[~df_filtered_cancer['Toma_mucho_Alcohol?'].isin(['No sabe/No está seguro', 'Se rehúsa a responder'])]

# Agrupar el DataFrame por 'Toma_mucho_Alcohol?' y contar las ocurrencias para cada tipo de cáncer
df_grouped = df_filteredT.groupby(['Que_tipo_de_cancer?', 'Toma_mucho_Alcohol?']).size().unstack(fill_value=0)

# Crear gráficos de tortas para cada tipo de cáncer
for cancer_type in valores_cancer:
    df_cancer_type = df_grouped.loc[cancer_type] if cancer_type in df_grouped.index else pd.Series()

    if not df_cancer_type.empty and df_cancer_type.sum() > 0:
        plt.figure(figsize=(8, 6))
        df_cancer_type.plot(kind='pie', autopct='%1.1f%%', startangle=90)
        plt.title(f'Distribución de Toma_mucho_Alcohol? para Tipo de Cáncer = {cancer_type}')
        plt.ylabel('')  # Ocultar la etiqueta del eje y
        plt.grid(axis='y', linestyle='--', alpha=0.7)
        plt.tight_layout()  # Ajustar el diseño para que todo se muestre bien
        plt.show()
    else:
        print(f'No hay datos para el Tipo de Cáncer = {cancer_type}')

No parece que la toma de mucho alcohol tenga efecto sobre el desarrollo de cancer.

In [ ]:
# Filtrar el DataFrame para excluir los valores 'No sabe/No está seguro' y 'Se rehúsa a responder'
df_filteredT = df_analisis[~df_analisis['Acitividad_Fisica?'].isin(['No se sabe', 'Se Rehusa a responder'])]

# Agrupar el DataFrame por 'TRANS?' y 'Tuvo_Cancer?', y contar las ocurrencias
df_grouped = df_filteredT.groupby(['Acitividad_Fisica?', 'Tuvo_Cancer?']).size().unstack(fill_value=0)

# Crear gráficos de tortas para cada combinación de 'Tuvo_Cancer?'
for category in df_grouped.index:
    plt.figure(figsize=(8, 6))
    df_grouped.loc[category].plot(kind='pie', autopct='%1.1f%%', startangle=90)
    plt.title(f'Distribución de Tuvo_Cancer? para Acitividad_Fisica? = {category}')
    plt.ylabel('')  # Ocultar la etiqueta del eje y
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.tight_layout()  # Ajustar el diseño para que todo se muestre bien
    plt.show()

    #Fuma?	Toma_mucho_Alcohol?	Acitividad_Fisica?

In [ ]:
# Valores específicos para 'Que_tipo_de_cancer?'
valores_cancer = ['Ninguno', 'Piel (no melanoma)', 'Mama', 'Melanoma', 'Próstata', 'Piel (no se sabe qué tipo)' , 'Pulmón', 'Garganta - faringe', 'Hígado']

# Filtrar el DataFrame para los valores específicos en 'Que_tipo_de_cancer?'
df_filtered_cancer = df_filtrado2[df_filtrado2['Que_tipo_de_cancer?'].isin(valores_cancer)]

# Filtrar el DataFrame para excluir los valores 'No sabe/No está seguro' y 'Se rehúsa a responder' en 'Fuma?'
df_filteredT = df_filtered_cancer[~df_filtered_cancer['Acitividad_Fisica?'].isin(['No sabe/No está seguro', 'Se rehúsa a responder'])]

# Agrupar el DataFrame por 'Acitividad_Fisica?' y contar las ocurrencias para cada tipo de cáncer
df_grouped = df_filteredT.groupby(['Que_tipo_de_cancer?', 'Acitividad_Fisica?']).size().unstack(fill_value=0)

# Crear gráficos de tortas para cada tipo de cáncer
for cancer_type in valores_cancer:
    df_cancer_type = df_grouped.loc[cancer_type] if cancer_type in df_grouped.index else pd.Series()

    if not df_cancer_type.empty and df_cancer_type.sum() > 0:
        plt.figure(figsize=(8, 6))
        df_cancer_type.plot(kind='pie', autopct='%1.1f%%', startangle=90)
        plt.title(f'Distribución de Acitividad_Fisica? para Tipo de Cáncer = {cancer_type}')
        plt.ylabel('')  # Ocultar la etiqueta del eje y
        plt.grid(axis='y', linestyle='--', alpha=0.7)
        plt.tight_layout()  # Ajustar el diseño para que todo se muestre bien
        plt.show()
    else:
        print(f'No hay datos para el Tipo de Cáncer = {cancer_type}')

Según los graficos la Acitividad_Fisica pareciera disminuir la probabilidad de cancer.

In [ ]:
# Filtramos solo las columnas numéricas
df_numerico = df_filtrado.select_dtypes(include=[int, float])

# Calculamos la matriz de correlación
correlation_matrix = df_numerico.corr()

# Creamos el mapa de calor
plt.figure(figsize=(20, 18))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1)
plt.title('Heatmap de Correlación entre todas las variables')
plt.show()

Segun este grafico de correlaciones parece haber correlacion entre Histerectomía y el consumo de alcohol y fumar.

In [ ]:
# Combinaciones a analizar
combinaciones = [
    ('Si', 'Si'),
    ('No', 'Si'),
    ('Si', 'No'),
    ('No', 'No')]
# Crear una figura con dos subgráficos
fig, axs = plt.subplots(1, 2, figsize=(16, 6))

# Valores específicos para 'Que_tipo_de_cancer?'
tipos_cancer = ['Útero/Uterino', 'Ninguno']

for i, tipo_cancer in enumerate(tipos_cancer):
    # Filtrar el DataFrame para el valor específico en 'Que_tipo_de_cancer?'
    df_filtered = df_filtrado2[df_filtrado2['Que_tipo_de_cancer?'] == tipo_cancer]

    # Filtrar el DataFrame para excluir los valores 'No sabe/No está seguro' y 'Se rehúsa a responder' en 'Fuma?' y 'Toma_mucho_Alcohol?'
    df_filtered = df_filtered[~df_filtered['Fuma?'].isin(['No sabe/No está seguro', 'Se rehúsa a responder'])]
    df_filtered = df_filtered[~df_filtered['Toma_mucho_Alcohol?'].isin(['No sabe/No está seguro', 'Se rehúsa a responder'])]

    # Contar las ocurrencias para las combinaciones deseadas
    conteo_combinaciones = []
    for toma_alcohol, fuma in combinaciones:
        count = df_filtered[
            (df_filtered['Toma_mucho_Alcohol?'] == toma_alcohol) &
            (df_filtered['Fuma?'] == fuma)
        ].shape[0]
        conteo_combinaciones.append(count)

    # Crear el gráfico de torta
    axs[i].pie(conteo_combinaciones,
              labels=[f'Toma={toma_alcohol}, Fuma={fuma}' for toma_alcohol, fuma in combinaciones],
              autopct='%1.1f%%', startangle=90,
              colors=plt.cm.Paired(range(len(combinaciones))))
    axs[i].set_title(f'Distribución de Combinaciones para Tipo de Cáncer = {tipo_cancer}')

plt.tight_layout()  # Ajustar el diseño para que todo se muestre bien
plt.show()

Pareciera que no hay un cambio apreciable en el cancer de Útero respecto a si toma y/o fuma o no lo hace, pero si filtramos solo para cuando el sexo es el femenino, lo cual deberia ser la unica forma de que haya este tipo de cancer, podemos observar:

In [ ]:
# Crear una función para generar gráficos de torta
def plot_pie_chart(df, tipo_cancer, title):
    # Filtrar el DataFrame para el valor específico en 'Que_tipo_de_cancer?'
    df_filtered = df[df['Que_tipo_de_cancer?'] == tipo_cancer]

    # Filtrar el DataFrame para el valor específico en 'Sexo'
    df_filtered = df_filtered[df_filtered['Sexo'] == 'Femenino']

    # Filtrar el DataFrame para excluir los valores 'No sabe/No está seguro' y 'Se rehúsa a responder' en 'Fuma?' y 'Toma_mucho_Alcohol?'
    df_filtered = df_filtered[~df_filtered['Fuma?'].isin(['No sabe/No está seguro', 'Se rehúsa a responder'])]
    df_filtered = df_filtered[~df_filtered['Toma_mucho_Alcohol?'].isin(['No sabe/No está seguro', 'Se rehúsa a responder'])]

    # Contar las ocurrencias para las combinaciones deseadas
    combinaciones = [
        ('Si', 'Si'),
        ('No', 'Si'),
        ('Si', 'No'),
        ('No', 'No')
    ]

    conteo_combinaciones = []
    for toma_alcohol, fuma in combinaciones:
        count = df_filtered[
            (df_filtered['Toma_mucho_Alcohol?'] == toma_alcohol) &
            (df_filtered['Fuma?'] == fuma)
        ].shape[0]
        conteo_combinaciones.append(count)

    # Crear el gráfico de torta
    plt.figure(figsize=(8, 6))
    plt.pie(conteo_combinaciones, labels=[f'Toma={toma_alcohol}, Fuma={fuma}' for toma_alcohol, fuma in combinaciones],
            autopct='%1.1f%%', startangle=90, colors=plt.cm.Paired(range(len(combinaciones))))
    plt.title(title)
    plt.tight_layout()  # Ajustar el diseño para que todo se muestre bien

    # Mostrar el gráfico
    plt.show()

# Valor específico para 'Que_tipo_de_cancer?' = 'Útero/Uterino'
plot_pie_chart(df_filtrado2, 'Útero/Uterino',
               'Distribución de Combinaciones de Toma_mucho_Alcohol? y Fuma? para Tipo de Cáncer = Útero/Uterino')

# Valor específico para 'Que_tipo_de_cancer?' = 'Ninguno'
plot_pie_chart(df_filtrado2, 'Ninguno',
               'Distribución de Combinaciones de Toma_mucho_Alcohol? y Fuma? para Tipo de Cáncer = Ninguno')


Mayor porcentaje de personas que si fuman.

# Preparación para los Modelos

En base a las variables, se probarán diferentes modelos para comprobar si efectivamente son funcionales y cual da mejores resultados.
Para este punto el dataframe esta bastante limitado ya que todos las columnas correspondientes al tipo de cáncer tienen sentido para predecir si tiene cáncer o no. Quizás si podrían utilizarse para predecir que tipo de cáncer tienen.

In [ ]:
df_analisis

Si bien inicialmente la tabla ya tenía valores numéricos originalmente, por más que sea redundante realizare un Encoder ya que en este caso me sirve de práctica.

In [ ]:
# label encoder para la categoricas
column=['Tuvo_Cancer?','Sexo','Fuma?','Toma_mucho_Alcohol?','Acitividad_Fisica?','Edad','TRANS?']

df_analisis[column] = df_analisis[column].apply(LabelEncoder().fit_transform)

In [ ]:
df_analisis

In [ ]:
# Escalar las variables numéricas 'IMC' y 'Edad'
scaler = StandardScaler()
df_analisis[['IMC', 'Edad']] = scaler.fit_transform(df_analisis[['IMC', 'Edad']])

Se separan los datos en variables objetivo (Y) y las variables explicatorias o características(X). Además se establece los subconjuntos de entrenamiento y de prueba. Estos se utilizan en todos los modelos para que la comparación sea correcta.

In [ ]:
# Separar las características (X) y la variable objetivo (y)
X = df_analisis.drop('Tuvo_Cancer?', axis=1)  # Características predictoras
y = df_analisis['Tuvo_Cancer?']  # Variable de respuesta

# Dividir el dataset en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Aclaro que tanto la implementación como los ajustes de los modelos son similares entre sí, por lo que solo aclaro la primera vez que se está haciendo y no lo mismo en cada modelo. Además la carga de librerías específicas para los modelos o ajustes los dejo en cada código de este apartado ya que una vez elegido el método no tendría sentido utilizar los demás, por lo que no sería necesario importar todas las librerías y de esta manera es más simple de tomar solo las librerías necesarias.

# Logistic Regression

Importar librería y creación y entrenamiento del modelo:

In [ ]:
from sklearn.linear_model import LogisticRegression

# Crear y entrenar el modelo de regresión logística
logreg = LogisticRegression()
logreg.fit(X_train, y_train)

Testeo y evaluación del modelo:

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

# Predicciones
y_pred = logreg.predict(X_test)

# Matriz de confusión
print("Matriz de confusión:")
print(confusion_matrix(y_test, y_pred))

# Informe de clasificación
print("Informe de clasificación:")
print(classification_report(y_test, y_pred))

Este resultado básicamente asume que ningún caso tiene cáncer. Esto se da porque la diferencia entre casos negativos y positivos es enorme.  Para esto aplico SMOTE, lo cual crea datos ficticios pero basado en los datos minoritarios(Cáncer positivo) para aumentar su número y disminuir la precisión:

In [63]:
from imblearn.over_sampling import SMOTE

# Aplicar SMOTE para balancear las clases
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

# Reentrenar el modelo con los datos balanceados
logreg = LogisticRegression()
logreg.fit(X_resampled, y_resampled)

# Evaluar nuevamente con el conjunto de prueba
y_pred = logreg.predict(X_test)

# Matriz de confusión y reporte de clasificación
print("Matriz de confusión:")
print(confusion_matrix(y_test, y_pred))

print("Informe de clasificación:")
print(classification_report(y_test, y_pred))

Matriz de confusión:
[[44455 30559]
 [ 1070  3182]]
Informe de clasificación:
              precision    recall  f1-score   support

           0       0.98      0.59      0.74     75014
           1       0.09      0.75      0.17      4252

    accuracy                           0.60     79266
   macro avg       0.54      0.67      0.45     79266
weighted avg       0.93      0.60      0.71     79266



Mejoro mucho, pero la precisión de la clase 1 es muy mala al igual que otros valores. Es por esto que se ajusta el umbral de decisión, esto cambia el límite que pone el algoritmo para definir si es una clase o la otra, lo cual afecta al recall y la precisión. Por ejemplo captare más verdaderos positivos pero tendré más falsos positivos.

In [64]:
y_pred_proba = logreg.predict_proba(X_test)[:, 1]
y_pred_adjusted = (y_pred_proba >= 0.3).astype(int)  # Ajusta el umbral aquí

print("Matriz de confusión:")
print(confusion_matrix(y_test, y_pred_adjusted))

print("Informe de clasificación:")
print(classification_report(y_test, y_pred_adjusted))

Matriz de confusión:
[[26124 48890]
 [  240  4012]]
Informe de clasificación:
              precision    recall  f1-score   support

           0       0.99      0.35      0.52     75014
           1       0.08      0.94      0.14      4252

    accuracy                           0.38     79266
   macro avg       0.53      0.65      0.33     79266
weighted avg       0.94      0.38      0.50     79266



In [65]:
# Ajustar el umbral a 0.4
threshold = 0.3
y_pred_adjusted = (y_pred_proba >= threshold).astype(int)

# Evaluar nuevamente con el umbral ajustado
print("Matriz de confusión:")
print(confusion_matrix(y_test, y_pred_adjusted))

print("Informe de clasificación:")
print(classification_report(y_test, y_pred_adjusted))


Matriz de confusión:
[[26124 48890]
 [  240  4012]]
Informe de clasificación:
              precision    recall  f1-score   support

           0       0.99      0.35      0.52     75014
           1       0.08      0.94      0.14      4252

    accuracy                           0.38     79266
   macro avg       0.53      0.65      0.33     79266
weighted avg       0.94      0.38      0.50     79266



# Decision Tree


In [66]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix

In [67]:
# Crear el modelo de árbol de decisión
tree_model = DecisionTreeClassifier(random_state=42, class_weight='balanced')

# Entrenar el modelo
tree_model.fit(X_train, y_train)

DecisionTreeClassifier(class_weight='balanced', random_state=42)

In [68]:
y_pred_tree = tree_model.predict(X_test)

In [69]:
print("Matriz de confusión:")
print(confusion_matrix(y_test, y_pred_tree))

print("Informe de clasificación:")
print(classification_report(y_test, y_pred_tree))

Matriz de confusión:
[[56489 18525]
 [ 2458  1794]]
Informe de clasificación:
              precision    recall  f1-score   support

           0       0.96      0.75      0.84     75014
           1       0.09      0.42      0.15      4252

    accuracy                           0.74     79266
   macro avg       0.52      0.59      0.49     79266
weighted avg       0.91      0.74      0.81     79266



En este caso se utiliza GridSearchCV cuya función es buscar los mejores hiperparámetros probando diferentes combinaciones. Esto tipo de análisis toma tiempo, por lo que el código aplicado lo comento y después se corre con los mejores hiperparámetros obtenidos.

In [70]:
#from sklearn.model_selection import GridSearchCV

#param_grid = {
#    'max_depth': [3, 5, 10, None],
#    'min_samples_split': [2, 10, 20],
#    'min_samples_leaf': [1, 5, 10]
#}

#grid_search = GridSearchCV(DecisionTreeClassifier(random_state=42, class_weight='balanced'), param_grid, cv=5, scoring='f1')
#grid_search.fit(X_train, y_train)

#print("Mejores parámetros:", grid_search.best_params_)


In [71]:
tree_model = DecisionTreeClassifier(random_state=42, class_weight='balanced', max_depth=5)


In [72]:
# Entrenar el modelo de árbol de decisión con los mejores parámetros
tree_model_optimized = DecisionTreeClassifier(random_state=42, class_weight='balanced',
                                              max_depth=5, min_samples_leaf=1, min_samples_split=2)
tree_model_optimized.fit(X_train, y_train)

# Realizar predicciones con el modelo optimizado
y_pred_tree_optimized = tree_model_optimized.predict(X_test)

# Evaluar el modelo optimizado
print("Matriz de confusión (Optimizado):")
print(confusion_matrix(y_test, y_pred_tree_optimized))

print("Informe de clasificación (Optimizado):")
print(classification_report(y_test, y_pred_tree_optimized))

Matriz de confusión (Optimizado):
[[43548 31466]
 [  880  3372]]
Informe de clasificación (Optimizado):
              precision    recall  f1-score   support

           0       0.98      0.58      0.73     75014
           1       0.10      0.79      0.17      4252

    accuracy                           0.59     79266
   macro avg       0.54      0.69      0.45     79266
weighted avg       0.93      0.59      0.70     79266



# Random Forest

In [73]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report

In [74]:
# Separar variables predictoras (X) y variable objetivo (y)
X = df_analisis.drop('Tuvo_Cancer?', axis=1)
y = df_analisis['Tuvo_Cancer?']

# Dividir el conjunto de datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Entrenar el modelo Random Forest
rf = RandomForestClassifier(random_state=42)
rf.fit(X_train, y_train)

# Hacer predicciones
y_pred = rf.predict(X_test)


In [75]:
# Matriz de confusión
print("Matriz de confusión:")
print(confusion_matrix(y_test, y_pred))

# Informe de clasificación
print("Informe de clasificación:")
print(classification_report(y_test, y_pred))


Matriz de confusión:
[[111083   1517]
 [  6139    159]]
Informe de clasificación:
              precision    recall  f1-score   support

           0       0.95      0.99      0.97    112600
           1       0.09      0.03      0.04      6298

    accuracy                           0.94    118898
   macro avg       0.52      0.51      0.50    118898
weighted avg       0.90      0.94      0.92    118898



In [76]:
#Se utilizo este codigo para buscar los parametros optimos, pero lo comento ya que tarda demasiado tiempo
# # Definir los hiperparámetros a optimizar
# param_grid = {
#     'n_estimators': [100, 200, 300],
#     'max_depth': [5, 10, 15],
#     'min_samples_split': [2, 5, 10],
#     'min_samples_leaf': [1, 2, 4]
# }

# # Usar GridSearchCV para buscar los mejores hiperparámetros
# rf_grid = GridSearchCV(estimator=RandomForestClassifier(random_state=42),
#                        param_grid=param_grid, cv=5, verbose=2, n_jobs=-1)

# # Entrenar el modelo optimizado
# rf_grid.fit(X_train, y_train)

# # Ver los mejores hiperparámetros
# print("Mejores parámetros:", rf_grid.best_params_)

# # Predecir con el modelo optimizado
# y_pred_optimizado = rf_grid.best_estimator_.predict(X_test)

# # Evaluar el modelo optimizado
# print("Matriz de confusión (Optimizado):")
# print(confusion_matrix(y_test, y_pred_optimizado))

# print("Informe de clasificación (Optimizado):")
# print(classification_report(y_test, y_pred_optimizado))


In [77]:
from imblearn.over_sampling import SMOTE

sm = SMOTE(random_state=42)
X_train_res, y_train_res = sm.fit_resample(X_train, y_train)


In [78]:
rf = RandomForestClassifier(n_estimators=100, max_depth=5, class_weight='balanced', random_state=42)
rf.fit(X_train, y_train)


RandomForestClassifier(class_weight='balanced', max_depth=5, random_state=42)

In [79]:
y_pred_prob = rf.predict_proba(X_test)[:, 1]
threshold = 0.3  # Ajustar el umbral
y_pred_adjusted = (y_pred_prob >= threshold).astype(int)

In [80]:
rf_balanced = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=42)
rf_balanced.fit(X_train_res, y_train_res)


RandomForestClassifier(max_depth=5, random_state=42)

In [81]:
y_pred_balanced = rf_balanced.predict(X_test)


In [82]:
from sklearn.metrics import confusion_matrix, classification_report

print("Matriz de confusión:")
print(confusion_matrix(y_test, y_pred_balanced))

print("Informe de clasificación:")
print(classification_report(y_test, y_pred_balanced))


Matriz de confusión:
[[57779 54821]
 [  938  5360]]
Informe de clasificación:
              precision    recall  f1-score   support

           0       0.98      0.51      0.67    112600
           1       0.09      0.85      0.16      6298

    accuracy                           0.53    118898
   macro avg       0.54      0.68      0.42    118898
weighted avg       0.94      0.53      0.65    118898



# XGBoost

In [ ]:
import xgboost as xgb
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report
from xgboost import XGBClassifier



In [87]:
X = df_analisis.drop('Tuvo_Cancer?', axis=1)
y = df_analisis['Tuvo_Cancer?']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
# Crear el modelo de XGBoost
xgb_model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss')


In [88]:
## Definir los parámetros a optimizar
#param_grid = {
#    'max_depth': [3, 5, 7],
#    'learning_rate': [0.01, 0.1, 0.3],
#    'n_estimators': [100, 200, 500],
#    'subsample': [0.7, 0.8, 1.0],
#    'colsample_bytree': [0.7, 0.8, 1.0]
#}
#{'colsample_bytree': 0.7, 'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 100, 'subsample': 0.7}

## Configurar GridSearchCV
#grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, cv=5, scoring='accuracy', verbose=1, n_jobs=-1)
#grid_search.fit(X_train, y_train)

# Imprimir los mejores parámetros
#print("Mejores parámetros: ", grid_search.best_params_)


In [89]:
#{'colsample_bytree': 0.7, 'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 100, 'subsample': 0.7}
## Predecir en el conjunto de prueba
#y_pred = grid_search.best_estimator_.predict(X_test)

## Matriz de confusión e informe de clasificación
#conf_matrix = confusion_matrix(y_test, y_pred)
#class_report = classification_report(y_test, y_pred)

#print("Matriz de confusión:")
#print(conf_matrix)
#print("\nInforme de clasificación:")
#print(class_report)



XGBClassifier mejora el rendimiento de xgboost

In [90]:
# Crear el modelo XGBoost con los mejores hiperparámetros
xgb_best_model = XGBClassifier(
    colsample_bytree=0.7,
    learning_rate=0.01,
    max_depth=3,
    n_estimators=100,
    subsample=0.7
)

# Entrenar el modelo con los datos de entrenamiento
xgb_best_model.fit(X_train, y_train)

# Predecir en el conjunto de prueba
y_pred = xgb_best_model.predict(X_test)

# Matriz de confusión e informe de clasificación
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

# Mostrar la matriz de confusión e informe de clasificación
print("Matriz de Confusión:")
print(conf_matrix)

print("\nInforme de Clasificación:")
print(class_report)

Matriz de Confusión:
[[112600      0]
 [  6298      0]]

Informe de Clasificación:
              precision    recall  f1-score   support

           0       0.95      1.00      0.97    112600
           1       0.00      0.00      0.00      6298

    accuracy                           0.95    118898
   macro avg       0.47      0.50      0.49    118898
weighted avg       0.90      0.95      0.92    118898



C:\Users\MATIAS\miniconda3\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\MATIAS\miniconda3\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\MATIAS\miniconda3\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [92]:

params = {
    'colsample_bytree': 0.7,
    'learning_rate': 0.01,
    'max_depth': 3,
    'n_estimators': 100,
    'subsample': 0.7,
    'scale_pos_weight': (len(y_train) - sum(y_train)) / sum(y_train)  # Ajustar el peso de la clase 1
}

# Reentrenar el modelo con los nuevos parámetros
xgb_model = XGBClassifier(**params)
xgb_model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.7, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.01, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [93]:
# Entrenamos el modelo con los mejores parámetros
xgb_model = XGBClassifier(colsample_bytree=0.7, learning_rate=0.01, max_depth=3, n_estimators=100, subsample=0.7)
xgb_model.fit(X_train, y_train)

# Hacemos predicciones en los datos de prueba
y_pred = xgb_model.predict(X_test)

# Evaluamos el rendimiento del modelo
print("Matriz de confusión:")
print(confusion_matrix(y_test, y_pred))

print("\nInforme de clasificación:")
print(classification_report(y_test, y_pred))


Matriz de confusión:
[[112600      0]
 [  6298      0]]

Informe de clasificación:
              precision    recall  f1-score   support

           0       0.95      1.00      0.97    112600
           1       0.00      0.00      0.00      6298

    accuracy                           0.95    118898
   macro avg       0.47      0.50      0.49    118898
weighted avg       0.90      0.95      0.92    118898



C:\Users\MATIAS\miniconda3\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\MATIAS\miniconda3\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\MATIAS\miniconda3\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [94]:
xgb_model = XGBClassifier(colsample_bytree=0.7, learning_rate=0.01, max_depth=3, n_estimators=100, subsample=0.7, scale_pos_weight=(len(y_train) - sum(y_train)) / sum(y_train))
xgb_model.fit(X_train, y_train)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.7, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.01, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [95]:
# Calcular el peso para balancear las clases
class_weight = (len(y_train) - sum(y_train)) / sum(y_train)

# Reentrenar el modelo con los nuevos parámetros, incluyendo scale_pos_weight
xgb_model = XGBClassifier(
    colsample_bytree=0.7,
    learning_rate=0.01,
    max_depth=3,
    n_estimators=100,
    subsample=0.7,
    scale_pos_weight=class_weight  # Peso de las clases
)

xgb_model.fit(X_train, y_train)

# Realizar las predicciones y evaluar el modelo
y_pred = xgb_model.predict(X_test)

# Matriz de confusión y reporte de clasificación
from sklearn.metrics import confusion_matrix, classification_report

print("Matriz de confusión:")
print(confusion_matrix(y_test, y_pred))

print("Informe de clasificación:")
print(classification_report(y_test, y_pred))


Matriz de confusión:
[[64188 48412]
 [ 1276  5022]]
Informe de clasificación:
              precision    recall  f1-score   support

           0       0.98      0.57      0.72    112600
           1       0.09      0.80      0.17      6298

    accuracy                           0.58    118898
   macro avg       0.54      0.68      0.44    118898
weighted avg       0.93      0.58      0.69    118898



In [97]:
from imblearn.over_sampling import SMOTE
from collections import Counter

# Ver la distribución de clases antes de SMOTE
print(f"Distribución de clases antes de SMOTE: {Counter(y_train)}")

# Definir el SMOTE
smote = SMOTE(sampling_strategy='auto')  # auto para generar un equilibrio

# Aplicar SMOTE al conjunto de entrenamiento
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

# Ver la distribución de clases después de SMOTE
print(f"Distribución de clases después de SMOTE: {Counter(y_resampled)}")

# Entrenar el modelo con los datos balanceados
xgb_model = XGBClassifier(
    colsample_bytree=0.7,
    learning_rate=0.01,
    max_depth=3,
    n_estimators=100,
    subsample=0.7
)

xgb_model.fit(X_resampled, y_resampled)

# Realizar las predicciones en el conjunto de prueba
y_pred = xgb_model.predict(X_test)

# Matriz de confusión y reporte de clasificación
print("Matriz de confusión:")
print(confusion_matrix(y_test, y_pred))

print("Informe de clasificación:")
print(classification_report(y_test, y_pred))


Distribución de clases antes de SMOTE: Counter({0: 263125, 1: 14303})
Distribución de clases después de SMOTE: Counter({0: 263125, 1: 263125})
Matriz de confusión:
[[64921 47679]
 [ 1315  4983]]
Informe de clasificación:
              precision    recall  f1-score   support

           0       0.98      0.58      0.73    112600
           1       0.09      0.79      0.17      6298

    accuracy                           0.59    118898
   macro avg       0.54      0.68      0.45    118898
weighted avg       0.93      0.59      0.70    118898



under_sampling sería como smote pero en lugar de aumentar la variable  minoritaria, disminuye la mayoritaria.

In [98]:
from imblearn.under_sampling import RandomUnderSampler


# Ver la distribución de clases antes del undersampling
print(f"Distribución de clases antes del undersampling: {Counter(y_train)}")

# Definir el undersampler
undersampler = RandomUnderSampler(sampling_strategy=1.0)  # 1.0 para balancear las clases 50-50

# Aplicar el undersampling al conjunto de entrenamiento
X_resampled, y_resampled = undersampler.fit_resample(X_train, y_train)

# Ver la distribución de clases después del undersampling
print(f"Distribución de clases después del undersampling: {Counter(y_resampled)}")

# Entrenar el modelo con los datos balanceados
xgb_model = XGBClassifier(
    colsample_bytree=0.7,
    learning_rate=0.01,
    max_depth=3,
    n_estimators=100,
    subsample=0.7,
    scale_pos_weight=1  # Ahora no se necesita ajustar el peso
)

xgb_model.fit(X_resampled, y_resampled)

# Realizar las predicciones en el conjunto de prueba
y_pred = xgb_model.predict(X_test)

# Matriz de confusión y reporte de clasificación
from sklearn.metrics import confusion_matrix, classification_report

print("Matriz de confusión:")
print(confusion_matrix(y_test, y_pred))

print("Informe de clasificación:")
print(classification_report(y_test, y_pred))



Distribución de clases antes del undersampling: Counter({0: 263125, 1: 14303})
Distribución de clases después del undersampling: Counter({0: 14303, 1: 14303})
Matriz de confusión:
[[63381 49219]
 [ 1231  5067]]
Informe de clasificación:
              precision    recall  f1-score   support

           0       0.98      0.56      0.72    112600
           1       0.09      0.80      0.17      6298

    accuracy                           0.58    118898
   macro avg       0.54      0.68      0.44    118898
weighted avg       0.93      0.58      0.69    118898



# Conclusión

El objetivo es detectar la mayor cantidad posible de la clase minoritaria (recall), entonces tanto XGBoost como Random Forest son las mejores opciones. De estos, XGBoost parece ofrecer un buen balance entre recall y f1-score.
Se podrían seguir optimizando estos pero la realidad es que debido a la naturaleza del problema y los datos, nunca se obtenga un buen resultado.
Hay que aclarar que si bien los factores analizados están relacionados con la aparición de cáncer, hay factores como la genética que no están contemplados aquí y podrían explicar mucho mejor la aparición de cáncer.

En resumen posiblemente se podría obtener mejores resultados tratando los datos de otra manera pero difícilmente pueda llegar a un buen resultado de predicción.

